In [ ]:
import torch
#from torch import nn
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as func
from labml_nn.diffusion.ddpm.unet import UNet
from DDPMfunctions import *
from plotting_functions import *
from torchvision.utils import save_image
import os

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
network = UNet(image_channels=3,n_channels=64)
model = network.to(device)

In [ ]:
#Choose the varianche scheduler that the model is trained on, we have incorporated the name of the varianche schedule in the title of the models
from Varianceschedule import *
T= 1000
schedule = 'cosine'
beta = varianceschedule(T, schedule, device)
alpha = 1.0-beta
alphabar = torch.cumprod(alpha,dim=0)
sqrt_alphabar_prod = alphabar ** 0.5
sqrtmin_alphabar = (1 - alphabar) ** 0.5


In [ ]:
#Load a pretrained model in
model.load_state_dict(torch.load("./model_saved_linear",weights_only=True))

In [ ]:
#Sample images and save them in a folder for FID calculations
# Directory to save generated samples
output_dir = "generated_samples_GOOD/epoch_cosine/"
os.makedirs(output_dir, exist_ok=True)
with torch.no_grad():
    #We have used 3000 since it takes a lot of time to sample however increasing the number of samples yields a more accurate FID calculations
    num_samples = 3000  
    batch_size = 100
    #model.eval()
    for i in range(0, num_samples, batch_size):
            xT = torch.randn(batch_size, 3, 32, 32).to(device)  
            for p in range(0,T-20)[::-1]:
                t = torch.full((batch_size,),p).long().to(device)
                eps_pred = model(xT,t)
                xT = backward_process(eps_pred,t[0],xT,alpha,beta,sqrtmin_alphabar,device)

            xT = ((xT + 1) / 2)  
            for j in range(xT.size(0)):
                if j == 0:
                    print(f"{i}")
                save_image(xT[j], os.path.join(output_dir, f"{i+j}.png"))

In [ ]:
#We calculate FID by using the pip package 
#pip install pytorch-fid
#Github repo: https://github.com/mseitzer/pytorch-fid
#One can calculate the FID by running the following command in the console:
#python -m pytorch_fid path/to/dataset1 path/to/dataset2

#Make sure to have both the samples saved in a folder and the cifar/mnist saved in a folder